This is for exporting ranking location posting.
Also, change the geocoordinate from 4326 to 3857.

In [4]:
import pandas as pd
import sqlite3

from pyproj import Proj, transform

In [2]:
def read_sql():
    df = __read_sql()
    df = __select_taipei_area(df)
    df = __clean_raw_dataframe(df)
    return df

# read raw data from sql
def __read_sql():
    con = sqlite3.connect('../data/InstagramPost.sqlite')
    con2 = sqlite3.connect('../data/InstagramPost2.sqlite')
    con3 = sqlite3.connect('../data/InstagramPost3.sqlite')
    SQL = pd.read_sql_query('select * from InstagramPost', con)
    SQL2 = pd.read_sql_query('select * from InstagramPost_table2', con)
    SQL3 = pd.read_sql_query('select * from InstagramPost_table3', con)
    SQL4 = pd.read_sql_query('select * from InstagramPost_table4', con)
    SQL5 = pd.read_sql_query('select * from InstagramPost_table2', con2)
    SQL6 = pd.read_sql_query('select * from InstagramPost', con3)

    # concat SQL tables
    SQL = pd.concat([SQL, SQL2, SQL3, SQL4, SQL5, SQL6]).drop_duplicates()
    del SQL2, SQL3, SQL4, SQL5, SQL6
    SQL['datetime'] = pd.to_datetime(SQL.date, unit='s')
    instagram2016 = SQL[SQL.datetime.dt.year == 2016]
    del SQL
    return instagram2016

# select taipei area
def __select_taipei_area(df):
#     # 基隆路附近
#     lat_min = 25.032840
#     lat_max = 25.051496
#     lng_min = 121.557441
#     lng_max = 121.580229
    
    # 台北市
    lat_min, lng_min = 24.975456, 121.461888
    lat_max, lng_max = 25.100281, 121.611286

    df['lat'] = df.lat.astype(float)
    df['lng'] = df.lng.astype(float)

    lat_bound = df.lat.between(lat_min, lat_max)
    lng_bound = df.lng.between(lng_min, lng_max)
    df = df[lat_bound & lng_bound]
    return df

# clean datafrome
def __clean_raw_dataframe(df):
    df = df[['location_id', 'location_name', 'lat', 'lng', 'caption', 'comments',
             'date', 'media_id', 'is_video', 'likes', 'owner_id', 'thumbnail_src',
             'display_src', 'datetime']].reset_index().drop(columns='index')
    return df

In [80]:
# proj from 4326 to 3857
def proj_transform(lat, lng):
    inProj = Proj(init='epsg:4326')
    outProj = Proj(init='epsg:3857')
    lng, lat = transform(inProj, outProj, lng, lat)
    return lat, lng

def df_change_proj(df):
    df['coor'] = df.apply(lambda df: proj_transform(df['lat'], df['lng']), axis=1)
    df['lat'] = df['coor'].apply(lambda x: x[0])
    df['lng'] = df['coor'].apply(lambda x: x[1])
    df = df.drop(columns=['coor'])
    return df

# 1. Read Instagram Data

In [3]:
ig2016 = read_sql()

In [81]:
loc_gb = ig2016.groupby('location_name')
rank = (pd.DataFrame(loc_gb.size())
            .reset_index()
            .rename(columns={0:'posts'})
            .sort_index(by='posts', ascending=False)
       )
# rank加上经纬度
loc_table = (ig2016.drop_duplicates('location_name')
                 .reset_index()
                 [['location_id', 'location_name', 'lat', 'lng']]
            )
rank = pd.merge(rank, loc_table, on='location_name')

D:\Users\ronn\Anaconda3\envs\nlp\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: by argument to sort_index is deprecated, please use .sort_values(by=...)
  """


In [82]:
rank.head()

,location_name,posts,location_id,lat,lng
0,TAIPEI 101 MALL 台北 101 購物中心,31126,213122053,25.033922,121.564453
1,BELLAVITA,24917,4816,25.039723,121.567416
2,松山文創園區 SCCP Taipei,22906,5027714,25.044063,121.557824
3,台北101觀景台,17607,1798709,25.033830,121.564676
4,ATT 4 FUN,16411,4612082,25.035327,121.565868


# 2. Projection change from 4326 to 3857

In [83]:
rank = df_change_proj(rank)

In [84]:
rank.head()

,location_name,posts,location_id,lat,lng
0,TAIPEI 101 MALL 台北 101 購物中心,31126,213122053,2.879912e+06,1.353249e+07
1,BELLAVITA,24917,4816,2.880624e+06,1.353282e+07
2,松山文創園區 SCCP Taipei,22906,5027714,2.881158e+06,1.353176e+07
3,台北101觀景台,17607,1798709,2.879901e+06,1.353252e+07
4,ATT 4 FUN,16411,4612082,2.880084e+06,1.353265e+07


rank.to_csv('location_ranking.csv', encoding='utf8')

# 2. Map Bounding

In [86]:
lat_min, lng_min = 24.975456, 121.461888
lat_max, lng_max = 25.100281, 121.611286
lat_min, lng_min = proj_transform(lat_min, lng_min)
lat_max, lng_max = proj_transform(lat_max, lng_max)
print('coor_min:  ', lat_min, ',', lng_min)
print('coor_max:  ', lat_max, ',', lng_max)

coor_min:   2872730.2479399117 , 13521075.522949627
coor_max:   2888066.921061524 , 13537706.43223516
